# Cooperação Internacional para o desenvolvimento do Complexo Econômico-Industrial Brasileiro
<p>Projeto Livre Reprograma curso Análise de Dados com Python </p>
<p>Aluna Cristiane Pereira</p>

<p><b>Introdução</b></p>
<p>Nesse notebook iremos mesclar e tratar as informações do CSV que será utilizado para responder as questões principais do projeto e gerar as primeiras visualizações.</p>
Perguntas e hipóteses exploratórias:
<ul>
<li>Quais são as condições da balança comercial em produtos de saúde para os países de interesse do estudo? Brasil, África do Sul, Índia, Argentina e México.</li>
<li>Quais produtos são importados/exportados por cada país?</li>
<li>Qual é a capacidade produtiva de IFA (Ingrediente Farmacêutico Ativo) de cada país?</li>
</ul>

<b>Importando os dados</b>

In [1]:
import pandas as pd

In [2]:
# Importando as bases de dados
df_Brazil_exports = pd.read_csv("Brazil-Pharmaceuticals-Exports.csv")
df_Brazil_imports = pd.read_csv("Brazil-Pharmaceuticals-Imports.csv")
df_Pharmaceutical_data = pd.read_csv("Pharmaceuticals-International-Trade-_parcial-data_.csv")

In [3]:
# Agora pretendo estudar como mesclar as bases de dados e preparar as colunas para as operações que desejo fazer
df_Pharmaceutical_data.columns

Index(['DESCRIPTION', 'COUNTRY', 'VALUE', 'YEAR', 'PARTNER', 'SOURCE'], dtype='object')

<b>Tratando os dados e dataframe final</b>

In [4]:
# Vou converter a coluna value para Float e a Year para datetime
# A conversão inicialmente deu errado, porque tem um dado incorreto na primeira linha do dataframe, vou substituí-lo, antes de converter
df_Pharmaceutical_data.iat[0,2] = "0.667"


In [5]:
#Verificando a alteração
df_Pharmaceutical_data.head(5)

,DESCRIPTION,COUNTRY,VALUE,YEAR,PARTNER,SOURCE
0,Urine bags imports (USD million),"South Africa, Eswatini and Kenya",0.667,2020.0,NaN,Medical Industries in Africa - A regional resp...
1,Imports medical technology %,South Africa,0.9,2018.0,NaN,Overview of the health technology sector in So...
2,Medical technological market (Rand billion),South Africa,350,2018.0,NaN,Overview of the health technology sector in So...
3,Medical technological market (Euros billion),South Africa,22,2018.0,NaN,Overview of the health technology sector in So...
4,Imports medical technology (Rand billion),South Africa,317,2018.0,NaN,Overview of the health technology sector in So...


In [6]:
# Deu um erro na conversão por causa de valores com vígula, ao invés de pontos, então voltei para substituir o caracter nas strings
df_Pharmaceutical_data['VALUE'] = df_Pharmaceutical_data['VALUE'].str.replace(',','.')

In [7]:
# Finalmente, a conversão para float
df_Pharmaceutical_data["VALUE"] = df_Pharmaceutical_data["VALUE"].astype(float)

In [16]:
# RETIREI PORQUE DEU ERRO NO TABLEAU
# Próximo passo, a conversão para datetime da coluna Year
# from datetime import datetime
# df_Pharmaceutical_data["YEAR"] = pd.to_datetime(df_Pharmaceutical_data["YEAR"])

In [8]:
# Agora vou explorar os demais dataframes
df_Brazil_exports.columns

Index(['Segmentos', 'Type', 'Year', 'Total exports million USD',
       'Main exports destiny', 'Value exports main partner',
       'Partner share (%)'],
      dtype='object')

In [9]:
# Verificando quais colunas vou precisar converter
df_Brazil_exports.dtypes

Segmentos                      object
Type                           object
Year                            int64
Total exports million USD       int64
Main exports destiny           object
Value exports main partner      int64
Partner share (%)             float64
dtype: object

In [10]:
# Quero alterar o nome da coluna Segmentos para o inglês
df_Brazil_exports.rename(columns={"Segmentos": "Segments"}, inplace="True")

In [20]:
# RETIREI PORQUE DEU ERRO NO TABLEAU
# Convertendo a colunar Year para datetime
# df_Brazil_exports["Year"] = pd.to_datetime(df_Brazil_exports["Year"])

In [11]:
# Explorando o último dataframe
df_Brazil_imports.columns

Index(['Segments', 'Type', 'Year', 'Total Imports million U$S',
       'Main imports origin', 'Value imports main partner',
       'Partner share (%)'],
      dtype='object')

In [12]:
df_Brazil_imports.dtypes

Segments                       object
Type                           object
Year                            int64
Total Imports million U$S     float64
Main imports origin            object
Value imports main partner      int64
Partner share (%)             float64
dtype: object

In [13]:
# RETIREI PORQUE DEU ERRO NO TABLEAU
# Preciso converter Year de int para datetime
# df_Brazil_imports['Year'] = pd.to_datetime(df_Brazil_imports['Year'])

In [14]:
#Agora irei unir os dataframes, desejo manter o máximo de informações possível, então vou fazer um merge outer
df_Brazil_pharmaceuticals = pd.merge(df_Brazil_exports, df_Brazil_imports, how = "outer")

In [25]:
df_Brazil_pharmaceuticals.to_csv("FinalBrazil_pharmaceuticals.csv")

In [15]:
# Deu erro no merge final por causa da ausência de colunas com valores semelhantes, então vou voltei para verificar possibilidades de intersecção
df_Brazil_pharmaceuticals.columns

Index(['Segments', 'Type', 'Year', 'Total exports million USD',
       'Main exports destiny', 'Value exports main partner',
       'Partner share (%)', 'Total Imports million U$S', 'Main imports origin',
       'Value imports main partner'],
      dtype='object')

In [16]:
# Notei que preciso adicionar uma coluna para identificar os dados do Brasil
df_Brazil_pharmaceuticals["COUNTRY"] = "Brazil"

In [17]:
# Decidi alterar também o nome da coluna Year, para ficar igual ao da outra tabela
df_Brazil_pharmaceuticals.rename(columns={"Year": "YEAR"}, inplace="True")

In [18]:
# Finalmente a intersecção entre todas as tabelas
df_Pharmaceutical_market = pd.merge(df_Brazil_pharmaceuticals, df_Pharmaceutical_data, how="outer")

In [19]:
# Verificando o dataframe final
df_Pharmaceutical_market.describe()

,YEAR,Total exports million USD,Value exports main partner,Partner share (%),Total Imports million U$S,Value imports main partner,VALUE
count,223.000000,51.000000,51.000000,101.000000,5.100000e+01,5.100000e+01,127.000000
mean,2020.309417,110092.411765,22752.156863,0.302921,3.628574e+05,8.061020e+04,8141.767441
std,1.345078,205128.542042,39548.599033,0.138388,1.128619e+06,2.234440e+05,30492.875329
min,2017.000000,1.000000,1.000000,0.107000,2.479000e+00,1.500000e+01,0.030000
25%,2020.000000,189.000000,38.000000,0.210000,2.305000e+02,9.050000e+01,4.479000
50%,2020.000000,17029.000000,5558.000000,0.269000,9.100000e+02,2.280000e+02,78.200000
75%,2021.000000,73532.000000,22209.000000,0.330000,1.134970e+05,4.162150e+04,425.000000
max,2022.000000,799762.000000,156420.000000,0.944000,6.689051e+06,1.304009e+06,180555.000000


In [20]:
#Exportando o dataframe final
df_Pharmaceutical_market.to_csv("FinalDataFramePharmaceuticalMarket.csv")

In [26]:
df_Pharmaceutical_market.head(20)

,Segments,Type,YEAR,Total exports million USD,Main exports destiny,Value exports main partner,Partner share (%),Total Imports million U$S,Main imports origin,Value imports main partner,COUNTRY,DESCRIPTION,VALUE,PARTNER,SOURCE
0,Total de Dispositivos Medicos (DMs),exports,2020.0,726.0,United States,186.0,0.256,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
1,Audiologia,exports,2020.0,5.0,United States,2.0,0.330,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
2,Cardiovascular,exports,2020.0,53.0,Switzerland,20.0,0.373,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
3,Demais equip. de uso hospitalar inclusive laser,exports,2020.0,63.0,United States,13.0,0.213,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
4,Diagnostico por imagem e seus insumos,exports,2020.0,27.0,United States,9.0,0.330,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
5,Equip. e material de apoio OPME,exports,2020.0,52.0,United States,24.0,0.456,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
6,Equipamentos para laboratorio,exports,2020.0,13.0,United States,4.0,0.346,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
7,Materiais e aparelhos para odontologia,exports,2020.0,53.0,United States,13.0,0.244,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
8,Materiais e suprimentos,exports,2020.0,222.0,United States,42.0,0.188,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
9,Oftalmologia,exports,2020.0,1.0,United States,1.0,0.583,NaN,NaN,NaN,Brazil,NaN,NaN,NaN,NaN
